In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
# 更改为每行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import numpy as np
def logprint(key,msg=None):
    type_key = type(key)
    if type_key ==str:
        shape = len(key)
    elif type_key ==np.ndarray:
        shape =key.shape
    else:
        shape =None
    if shape !=None:
        print(("logprint\tmsg:"+str(msg)+"\ttype:"+str(type(key))+"\tshape:"+str(shape)))
    else:
        print(("logprint\tmsg:"+str(msg)+"\ttype:"+str(type(key))))
    print(key)
tf.__version__

'1.6.0'

In [8]:
state = tf.Variable(0,name = 'counter')
one = tf.constant(1)
new_value = tf.add(state,one)
update = tf.assign(state,new_value)
init = tf.initialize_all_variables()

In [18]:
with tf.Session() as sess:
    sess.run(init)
#     print(sess.run(new_value))
    print((sess.run(state)))
    for _ in range(3):
        sess.run(update)
        print(sess.run(new_value))
        print((sess.run(state)))

0
2
1
3
2
4
3


TensorFlow操作（也简称为**ops**）可以接受任意数量的输入和产生任意数量的输出。例如，
* 加法和乘法运算每个都需要两个输入并产生一个输出。
* 常量和变量不需要输入（它们被称为**源操作**）。
* 输入和输出是多维数组，称为**张量**（因此称为“张量流”）。

就像NumPy数组一样，张量具有类型和形状。实际上，**在Python API中，张量只是由NumPy ndarrays表示**。它们通常包含浮点数，但你也可以使用它们来携带字符串（任意字节数组）。

在到目前为止的示例中，张量只包含一个标量值，但你当然可以对任何形状的数组执行计算。例如，以下代码操作2D数组以对California住房数据集执行**线性回归**（在第2章中介绍）。

* 它首先获取数据集;
* 然后它为所有训练实例添加了一个**额外的偏置输入特征**($x_0 = 1$)（它使用NumPy这样做，因此它立即运行）;
* 最后它创建两个TensorFlow常量节点 X 和 y 来保存这些数据和目标，它使用TensorFlow提供的一些矩阵运算来定义**theta**。

这些矩阵函数—**transpose()，matmul()和matrix_inverse()** —是一目了然的，但通常它们不会立即执行任何计算;相反，
* 他们在图表中创建节点，在运行图表时执行它们。你可以认识到 $θ$ 的定义对应于法线方程($θ= X^T·X^{-1}·X^T·y $；见第4章)。
* 最后，代码创建一个会话并使用它来评估**theta**

In [22]:
# reset_graph()
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
x = tf.constant(housing_data_plus_bias,dtype= tf.float32,name= 'X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y") 
XT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(x,x,transpose_a=True)),x,transpose_b=True),y)

In [20]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

使用优化器

In [30]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr" 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
# reset_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
##########################################################
##手动计算梯度
# gradients = 2/m * tf.matmul(tf.transpose(X), error)


# training_op = tf.assign(theta, theta - learning_rate * gradients)
##########################################################
##########################################################
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

optimizer = tf.train.MomentumOptimizer(learning_rate =learning_rate,momentum =0.9)
training_op = optimizer.minimize(mse)
##########################################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:# 每100次迭代打印出当前的均方误差（mse）
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.52731586
Epoch 200 MSE = 0.52441466
Epoch 300 MSE = 0.52432835
Epoch 400 MSE = 0.5243219
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.52432066
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.52432126
Epoch 900 MSE = 0.524321


In [2]:
import tensorflow as tf
A = tf.Variable([[0.8, 0.4, 0.5, 0.6],[0.1, 0.9, 0.2, 0.4],[0.1, 0.9, 0.4, 0.2]])
B = tf.Variable([1, 1, 2])
result = tf.nn.in_top_k(A, B, 2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(A))
    print(sess.run(B))
    print(sess.run(result))
#   k=1 [False True False]
#   k=2 [False True True]

C:\Users\persp\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[[0.8 0.4 0.5 0.6]
 [0.1 0.9 0.2 0.4]
 [0.1 0.9 0.4 0.2]]
[1 1 2]
[False  True  True]


placeholder